<a href="https://colab.research.google.com/github/Shrinjita/lg-help-webscraper/blob/main/WebscrapingProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install google-colab-selenium

In [ ]:
import os
import re
import time
import json
import csv
import pandas as pd
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import google_colab_selenium as gs

In [ ]:
# Initialize folder for the project
project_folder = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject"
os.makedirs(project_folder, exist_ok=True)

In [ ]:
# Initialize WebDriver
def init_driver():
    return gs.Chrome()

driver = init_driver()

# Scrape a single page for links
def scrape_page(driver, output_file):
    links = driver.find_elements(By.CSS_SELECTOR, 'a.MuiTypography-root.MuiTypography-subtitle2.MuiTypography-alignLeft.MuiLink-root.MuiLink-underlineHover.css-15k6acr')
    with open(output_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for link in links:
            title = link.text.strip()
            url = link.get_attribute('href')
            writer.writerow([title, url])
    print(f"Scraped {len(links)} links from the current page.")

# Navigate to the next page
def go_to_next_page(driver):
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Go to next page"]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        driver.execute_script("arguments[0].click();", next_page_button)
        print("Navigated to the next page.")
        return True
    except Exception as e:
        print(f"Error navigating to the next page: {e}")
        return False

# Main scraping loop
def scrape_links():
    url = "https://www.lg.com/us/support/help-library"
    driver.get(url)

    output_file = os.path.join(project_folder, "lg_help_links.csv")
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Title", "Link"])

    page_number = 1
    while True:
        print(f"Scraping page {page_number}...")
        scrape_page(driver, output_file)
        page_number += 1
        if not go_to_next_page(driver):
            print("No more pages to scrape or error in finding the next page button.")
            break
        time.sleep(3)

    driver.quit()

# Start scraping
scrape_links()


Initializing WebDriver...


<IPython.core.display.Javascript object>

Scraping page 1...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 2...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 3...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 4...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 5...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 6...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 7...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 8...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 9...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 10...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 11...
Scraped 5 links from the current page.
Navigated to the next page.
Scraping page 12...
Scraped 5 links from the current

In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links.csv"
df = pd.read_csv(file_path)

# Remove duplicate links
df_unique = df.drop_duplicates(subset='Link')

# Save the changes back to the same CSV file
df_unique.to_csv(file_path, index=False)

In [ ]:
import pandas as pd
import google_colab_selenium as gs
from selenium.webdriver.common.by import By
import time

# Initialize WebDriver
def initialize_driver():
    return gs.Chrome()

# Function to process a given DataFrame of links
def process_links(df, output_csv, classes_to_check):
    # Add columns for each class in `classes_to_check`
    for class_name in classes_to_check:
        df[class_name] = False  # Default to False

    # Initialize the WebDriver
    driver = initialize_driver()

    try:
        for index, row in df.iterrows():
            driver.get(row['Link'])
            time.sleep(5)
            try:
                article_left = driver.find_element(By.CLASS_NAME, "article-left")
                li_elements = article_left.find_elements(By.CLASS_NAME, "article-topic")
                theme_text = ", ".join([li.text.strip() for li in li_elements if li.text.strip()])
                df.at[index, "Theme"] = theme_text
            except Exception as e:
                print(f"Error extracting 'Theme': {e}")
                df.at[index, "Theme"] = ""
            for class_name in classes_to_check:
                try:
                    elements = driver.find_elements(By.CLASS_NAME, class_name)
                    df.at[index, class_name] = bool(elements)  # True if elements are found
                except Exception as e:
                    print(f"Error checking class '{class_name}': {e}")

    finally:
        driver.quit()  # Close the driver

    # Save the updated DataFrame to a CSV
    df.to_csv(output_csv, index=False)
    print(f"Data saved to {output_csv}")
    return df
# Input file path
input_csv = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links.csv"

# Read the input file
print(f"Reading input file: {input_csv}")
input_data = pd.read_csv(
    input_csv,
    on_bad_lines="skip"  # Skip rows with parsing errors.
)

# Split the data into two halves
mid_point = len(input_data) // 2
df_part1 = input_data.iloc[:mid_point].copy()  # First half
df_part2 = input_data.iloc[mid_point:].copy()  # Second half

print(f"Data split into two halves. First half: {len(df_part1)} rows, Second half: {len(df_part2)} rows.")
# Output CSV for the first half
output_csv_part1 = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links_processed_part1.csv"

# Classes to check on the webpag
classes_to_check = [
    "lgeai-error-code-item"
]

# Process the first half of the data
df_part1 = process_links(df_part1, output_csv_part1, classes_to_check)

Reading input file: /content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links.csv
Data split into two halves. First half: 334 rows, Second half: 335 rows.


<IPython.core.display.Javascript object>

Data saved to /content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links_processed_part1.csv


In [ ]:
# Output CSV for the second half
output_csv_part2 = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links_processed_part2.csv"

# Process the second half of the data
df_part2 = process_links(df_part2, output_csv_part2, classes_to_check)


<IPython.core.display.Javascript object>

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=38103): Read timed out. (read timeout=120)

In [ ]:
# Final output file
final_output_csv = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links_processed_final.csv"

# Combine the two parts
df_part1 = pd.read_csv(output_csv_part1)
df_part2 = pd.read_csv(output_csv_part2)

# Concatenate the DataFrames
df_final = pd.concat([df_part1, df_part2], ignore_index=True)

# Save the combined DataFrame to a CSV
df_final.to_csv(final_output_csv, index=False)
print(f"Final processed data saved to {final_output_csv}")


In [ ]:
# Input file path
input_csv = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links.csv"

# Read the input file
print(f"Reading input file: {input_csv}")
input_data = pd.read_csv(input_csv)

# Split the data into two halves
mid_point = len(input_data) // 2
df_part1 = input_data.iloc[:mid_point].copy()  # First half
df_part2 = input_data.iloc[mid_point:].copy()  # Second half

print(f"Data split into two halves. First half: {len(df_part1)} rows, Second half: {len(df_part2)} rows.")


In [ ]:
import pandas as pd

# Specify the path to the CSV file
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/lg_help_links_processed.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Check for initial data overview
print("Initial data overview:")
print(df.head())

# Create a boolean mask where 'Content' contains the 'Link' value
print("Creating a mask to identify rows where 'Link' is in 'Content'...")
mask = df.apply(lambda row: str(row['Link']) in str(row['Content']), axis=1)

# Drop rows where the mask is True
df_cleaned = df[~mask]

# Save the cleaned DataFrame to a new CSV
cleaned_csv_path = '/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/data_cleaned.csv'
df_cleaned.to_csv(cleaned_csv_path, index=False)

# Print the cleaned DataFrame or check the first few rows
print(f"Cleaned data saved to {cleaned_csv_path}")
print("Cleaned data overview:")
print(df_cleaned.head())


In [ ]:
# Folder to save images
image_folder = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/images"
os.makedirs(image_folder, exist_ok=True)

# Function to initialize the selenium driver
def init_driver():
    print("Initializing WebDriver...")
    driver = gs.Chrome()  # Correct method to start the WebDriver
    return driver

# Function to sanitize filenames
def sanitize_filename(filename):
    return re.sub(r'[\/:*?"<>|]', '_', filename)

# Function to extract data based on element class
def extract_element_text(driver, element_class, multiple=False, attribute=None):
    elements = driver.find_elements(By.CLASS_NAME, element_class)
    if multiple:
        return [el.get_attribute(attribute) if attribute else el.text.strip() for el in elements if el]
    return elements[0].text.strip() if elements else None

# Function to scrape data from a page
def scrape_site(url, driver=None):
    output_data = {
        "Theme": None,
        "Category": None,
        "Subclass_Category": []
    }

    try:
        driver.get(url)
        time.sleep(2)  # Allow page to load

        # Extract Theme
        output_data["Theme"] = extract_element_text(driver, "article-topic")

        # Extract Category
        output_data["Category"] = extract_element_text(driver, "article-category")

        # Extract Subclass Category
        subclass_elements = driver.find_elements(By.CLASS_NAME, "lgeai-box-hover")
        for element in subclass_elements:
            aria_label = element.get_attribute("aria-label")
            if aria_label:
                output_data["Subclass_Category"].append(aria_label)

        # Extract and group text content
        text_elements = driver.find_elements(By.CLASS_NAME, "lgeai-description")
        text_grouped = [", ".join([text.strip() for text in el.text.split('\n')]) for el in text_elements]

        # Extract video links
        video_links = [
            anchor.get_attribute("href")
            for anchor in driver.find_elements(By.TAG_NAME, "a")
            if "youtu.be" in (anchor.get_attribute("href") or "")
        ]

        # Download images
        image_data = []
        img_tags = driver.find_elements(By.CSS_SELECTOR, "div.img-only img")
        for img in img_tags:
            img_url = img.get_attribute("src")
            alt_text = img.get_attribute("alt") or "default_image"
            if img_url and alt_text:
                img_filename = os.path.join(image_folder, f"{sanitize_filename(alt_text)}.jpg")
                response = requests.get(img_url, stream=True)
                if response.status_code == 200:
                    with open(img_filename, "wb") as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)
                    image_data.append({"alt_text": alt_text, "img_url": img_url, "img_filename": img_filename})

        output_data["Content"] = {
            "text_grouped": text_grouped,
            "video_links": video_links,
            "images": image_data
        }

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    return output_data

# Save data as JSON file
def save_data_as_json(data, output_filename):
    output_json_filename = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/output_data.json"
    with open(output_json_filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    print(f"Data saved as {output_json_filename}")

# Main function to process URLs
def process_urls():
    # Path to the CSV file containing URLs
    csv_file = "/content/drive/MyDrive/Colab Notebooks/WebscrapingProject/data_cleaned.csv"

    # Initialize WebDriver once
    driver = init_driver()

    # Load data from CSV file
    df = pd.read_csv(csv_file)
    urls = df["Content"].dropna().tolist()  # Fetch links from the 'Content' column

    all_data = []

    # Loop through each URL and scrape data
    for index, row in df.iterrows():
        url = row["Content"]
        question = row["Title"]
        error_code = row["Error Code"]

        print(f"Scraping URL: {url}")
        scraped_data = scrape_site(url, driver=driver)

        # Prepare the final output data
        data = {
            "Question": question,
            "Theme": scraped_data["Theme"],
            "Category": scraped_data["Category"],
            "Subclass_Category": scraped_data["Subclass_Category"],
            "Error Code": error_code,
            "Content": scraped_data["Content"]
        }

        all_data.append(data)

    # Save all the data as a JSON file
    save_data_as_json(all_data, "output_data.json")

    # Quit the WebDriver once done
    driver.quit()

# Run the processing function
process_urls()
